<a href="https://colab.research.google.com/github/MokshagnaAnurag/Handwritten-Words-Recognition/blob/main/Handwritten_Words_Recongnition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install rarfile


In [6]:
import rarfile
import os
rar_path = "/content/data.rar"
extract_path = "/content/data"
with rarfile.RarFile(rar_path, 'r') as rf:
    rf.extractall(extract_path)

print(f"Files extracted to: {extract_path}")

Files extracted to: /content/data


In [36]:
import os
import cv2
import numpy as np

def load_images_and_labels(data_path):
    images = []
    labels = []

    for root, dirs, files in os.walk(data_path):
        for file in files:
            if file.endswith(('.png', '.jpg', '.jpeg')):
                img_path = os.path.join(root, file)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                img = cv2.resize(img, (64, 64))

                label = os.path.basename(root)

                images.append(img)
                labels.append(label)


    images = np.array(images, dtype='float32').reshape(-1, 64, 64, 1)
    labels = np.array(labels)

    return images, labels
data_path = "/content/data"
X, y = load_images_and_labels(data_path)
print(f"Loaded {len(X)} images with {len(set(y))} unique labels.")


Loaded 4355 images with 12 unique labels.


In [19]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
X = X / 255
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)

print(f"Label mapping: {dict(zip(label_encoder.classes_, range(len(label_encoder.classes_))))}")

Label mapping: {'BOXING': 0, 'FIVE': 1, 'JUMP': 2, 'QUICKLY': 3, 'THE': 4, 'WIZARDS': 5, 'boxing': 6, 'five': 7, 'jump': 8, 'quickly': 9, 'the': 10, 'wizards': 11}


In [26]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

def create_model(input_shape, num_classes):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.3))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

model = create_model(input_shape, num_classes)
history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    batch_size=32,
                    epochs=20,
                    verbose=1)


Epoch 1/20
109/109 ━━━━━━━━━━━━━━━━━━━━ 22s 189ms/step - accuracy: 0.0902 - loss: 2.4823 - val_accuracy: 0.0987 - val_loss: 2.4734
Epoch 2/20
109/109 ━━━━━━━━━━━━━━━━━━━━ 41s 193ms/step - accuracy: 0.0873 - loss: 2.4756 - val_accuracy: 0.1114 - val_loss: 2.4735
Epoch 3/20
109/109 ━━━━━━━━━━━━━━━━━━━━ 42s 205ms/step - accuracy: 0.0885 - loss: 2.4747 - val_accuracy: 0.0987 - val_loss: 2.4672
Epoch 4/20
109/109 ━━━━━━━━━━━━━━━━━━━━ 40s 200ms/step - accuracy: 0.0995 - loss: 2.4758 - val_accuracy: 0.0987 - val_loss: 2.4672
Epoch 5/20
109/109 ━━━━━━━━━━━━━━━━━━━━ 42s 208ms/step - accuracy: 0.0905 - loss: 2.4755 - val_accuracy: 0.0987 - val_loss: 2.4679
Epoch 6/20
109/109 ━━━━━━━━━━━━━━━━━━━━ 40s 202ms/step - accuracy: 0.0874 - loss: 2.4746 - val_accuracy: 0.0987 - val_loss: 2.4665
Epoch 7/20
109/109 ━━━━━━━━━━━━━━━━━━━━ 41s 203ms/step - accuracy: 0.0936 - loss: 2.4775 - val_accuracy: 0.0987 - val_loss: 2.4674
Epoch 8/20
109/109 ━━━━━━━━━━━━━━━━━━━━ 20s 188ms/step - accuracy: 0.0937 - loss: 2